<a href="https://colab.research.google.com/github/vilsonrodrigues/DataScienceEduc/blob/master/Analises_das_Tabelas_de_Notas_de_Lista_e_Prova%2C_e_Submissoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Juntando as tabelas nota de provas, nota de listas e submissoes em uma so

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from csv import reader

Import CSV

In [0]:
notap=pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopUsuarioNotaProva.csv")
notap=notap.drop(columns=["Unnamed: 0","nome","Unnamed: 0","ano"])

notal=pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/tabelaNotasListas.csv")
notal=notal.drop(columns=["descricaoTurma"])

sub=pd.read_csv("https://github.com/repitta/CienciaDeDadosEducacionais/raw/master/arquivosCSV/lopUseTurmasSubLista.csv")
sub=sub.drop(columns=["data","Unnamed: 0","idAluno","idLista","descricaoTurma","ano","nome","quantidade","porcentagemAcerto","idTurma"])


Substituindo valores nulors por zero

In [0]:
sub.replace(np.nan, 0, inplace=True)
notap.replace(np.nan, 0, inplace=True)
notal.replace(np.nan, 0, inplace=True)

Removendo as provas que contem video ou sao de testes

In [0]:
notap=notap.sort_values(by="idTurma")
notap["tituloProva"]=notap.tituloProva.astype(str)
notap=notap[(~notap["tituloProva"].str.contains("V")) & (~notap["tituloProva"].str.contains("este"))]

Fazendo consulta e dividindo por notas de primeira e segunda unidade, e depois juntando 

In [61]:
notap1=notap.loc[(notap["tituloProva"].str.contains("Prova 01")) | (notap["tituloProva"].str.contains("Prova 1")) | (notap["tituloProva"].str.contains("Prova de Laboratório 1")),["idAluno","idTurma","matricula","notaProva","questoesFeitasProva","quantidadeProva","tituloProva"]]
notap1.rename(columns={"notaProva":"notaProva1","questoesFeitasProva":"questoesFeitasProva1","quantidadeProva":"quantidadeProva1","tituloProva":"tituloProva1"},inplace=True)
notap2=notap.loc[(notap["tituloProva"].str.contains("Prova 02")) | (notap["tituloProva"].str.contains("Prova 2")) | (notap["tituloProva"].str.contains("Prova de Laboratório 2")) | (notap["tituloProva"].str.contains("Segunda")) | (notap["tituloProva"].str.contains("2ª")),["matricula","notaProva","questoesFeitasProva","quantidadeProva","tituloProva"]]    
notap2.rename(columns={"notaProva":"notaProva2","questoesFeitasProva":"questoesFeitasProva2","quantidadeProva":"quantidadeProva2","tituloProva":"tituloProva2"},inplace=True)

notaProva=pd.merge(right = notap2, left = notap1, on="matricula", how="outer")
notaProva.head()

,idAluno,idTurma,matricula,notaProva1,questoesFeitasProva1,quantidadeProva1,tituloProva1,notaProva2,questoesFeitasProva2,quantidadeProva2,tituloProva2
0,5991cb910be9d42f2f68b906,598e15296d8650eb27d52e3d,20170039453,72.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,14.285714,1.0,7.0,Prova 2 - Turma 01 - 2017.2
1,599b0563d9b4c2079007acc9,598e15296d8650eb27d52e3d,20170031473,44.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,47.571429,7.0,7.0,Prova 2 - Turma 01 - 2017.2
2,599b053e495fac076a06d3f7,598e15296d8650eb27d52e3d,20170036872,60.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,9.428571,7.0,7.0,Prova 2 - Turma 01 - 2017.2
3,599b053410b11c07650122e9,598e15296d8650eb27d52e3d,20170032274,32.0,4.0,5.0,Prova 1 - Turma 01 - 2017.2,80.857143,7.0,7.0,Prova 2 - Turma 01 - 2017.2
4,599b0522495fac076a06d3f6,598e15296d8650eb27d52e3d,20170039972,24.0,4.0,5.0,Prova 1 - Turma 01 - 2017.2,28.571429,6.0,7.0,Prova 2 - Turma 01 - 2017.2


Criando 5 DF para guardar o numero de questao submetidas de cada lista disponivel no LOP

In [64]:
sub["titulo"]=sub.titulo.astype(str)
sub=sub[(~sub["titulo"].str.contains("Prova"))]

sub1 = sub.loc[sub["titulo"].str.contains(" 1",na = False)]
sub1 = sub1.groupby(["matricula"])["questao"].count().reset_index().sort_values(by="matricula")
sub1.rename(columns={"questao":"qsub1"},inplace=True)

sub2 = sub.loc[sub["titulo"].str.contains(" 2",na = False)]
sub2 = sub2.groupby(["matricula"])["questao"].count().reset_index().sort_values(by="matricula")
sub2.rename(columns={"questao":"qsub2"},inplace=True)

sub3 = sub.loc[sub["titulo"].str.contains(" 3",na = False)]
sub3 = sub3.groupby(["matricula"])["questao"].count().reset_index().sort_values(by="matricula")
sub3.rename(columns={"questao":"qsub3"},inplace=True)

sub4 = sub.loc[(sub["titulo"].str.contains(" 4",na = False)) | (sub["titulo"].str.contains("4",na = False))]
sub4 = sub4.groupby(["matricula"])["questao"].count().reset_index().sort_values(by="matricula")
sub4.rename(columns={"questao":"qsub4"},inplace=True)

sub5 = sub.loc[sub["titulo"].str.contains(" 5",na = False)]
sub5 = sub5.groupby(["matricula"])["questao"].count().reset_index().sort_values(by="matricula")
sub5.rename(columns={"questao":"qsub5"},inplace=True)

sub1.head()

,matricula,qsub1
0,0.000000e+00,2011
1,2.014066e+09,1
2,2.014082e+09,9
3,2.014085e+09,1
4,2.014085e+09,12


Juntando as tabelas das listas

In [65]:
subm=pd.merge((pd.merge(pd.merge(sub1,sub2,on="matricula"),pd.merge(sub3,sub4,on="matricula"),on="matricula")),sub5,on="matricula")
subm.head()

,matricula,qsub1,qsub2,qsub3,qsub4,qsub5
0,0.000000e+00,2011,1464,910,243,170
1,2.014082e+09,9,6,7,18,14
2,2.014085e+09,1,7,4,1,22
3,2.015008e+09,5,19,16,9,8
4,2.015010e+09,59,22,42,26,21


In [66]:
notaProva.head()

,idAluno,idTurma,matricula,notaProva1,questoesFeitasProva1,quantidadeProva1,tituloProva1,notaProva2,questoesFeitasProva2,quantidadeProva2,tituloProva2
0,5991cb910be9d42f2f68b906,598e15296d8650eb27d52e3d,20170039453,72.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,14.285714,1.0,7.0,Prova 2 - Turma 01 - 2017.2
1,599b0563d9b4c2079007acc9,598e15296d8650eb27d52e3d,20170031473,44.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,47.571429,7.0,7.0,Prova 2 - Turma 01 - 2017.2
2,599b053e495fac076a06d3f7,598e15296d8650eb27d52e3d,20170036872,60.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,9.428571,7.0,7.0,Prova 2 - Turma 01 - 2017.2
3,599b053410b11c07650122e9,598e15296d8650eb27d52e3d,20170032274,32.0,4.0,5.0,Prova 1 - Turma 01 - 2017.2,80.857143,7.0,7.0,Prova 2 - Turma 01 - 2017.2
4,599b0522495fac076a06d3f6,598e15296d8650eb27d52e3d,20170039972,24.0,4.0,5.0,Prova 1 - Turma 01 - 2017.2,28.571429,6.0,7.0,Prova 2 - Turma 01 - 2017.2


Juntando todas as tabelas e fazendo um dado final

In [68]:
table=pd.merge(pd.merge(notal,notaProva,on="matricula"),subm,on="matricula")
table=table.drop(columns=["Unnamed: 0"])
table.head()

,matricula,nome,ano,lab1,qLab1,lab2,qLab2,lab3,qLab3,lab4,qLab4,lab5,qLab5,exe1,qexe1,exe2,qexe2,exe3,qexe3,exe4,qexe4,exe4.1,qexe4.1,exe5,qexe5,idAluno,idTurma,notaProva1,questoesFeitasProva1,quantidadeProva1,tituloProva1,notaProva2,questoesFeitasProva2,quantidadeProva2,tituloProva2,qsub1,qsub2,qsub3,qsub4,qsub5
0,20170106538,ALEFF RAMON DA SILVA COSTA,2017.2,0.0,1.0,10.0,5.0,4.0,2.0,6.0,3.0,4.0,3.0,0.666667,2.0,6.755556,13.0,3.993333,7.0,4.375,7.0,0.0000,0.0,2.500,5.0,599b051d495fac076a06d3f5,598e15296d8650eb27d52e3d,52.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,0.000000,1.0,7.0,Prova 2 - Turma 01 - 2017.2,6,28,10,10,12
1,20170039453,AMANDA,2017.2,10.0,5.0,10.0,5.0,10.0,5.0,10.0,5.0,10.0,5.0,7.333333,11.0,10.000000,18.0,10.000000,15.0,10.000,16.0,6.2500,5.0,10.000,16.0,5991cb910be9d42f2f68b906,598e15296d8650eb27d52e3d,72.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,14.285714,1.0,7.0,Prova 2 - Turma 01 - 2017.2,21,26,21,27,23
2,20170044776,BRUNA ALICE OLIVEIRA DE BRUNA,2017.2,4.0,2.0,2.0,1.0,2.0,1.0,4.0,2.0,2.0,1.0,1.333333,2.0,0.555556,1.0,1.333333,2.0,3.125,5.0,0.0000,0.0,0.625,1.0,5991cd76b3c8b42f3d5da404,598e15296d8650eb27d52e3d,36.0,3.0,5.0,Prova 1 - Turma 01 - 2017.2,18.857143,4.0,7.0,Prova 2 - Turma 01 - 2017.2,4,2,4,7,2
3,20170031473,BRUNA DE SOUZA PEREIRA,2017.2,10.0,5.0,10.0,5.0,4.0,2.0,4.0,2.0,0.0,2.0,3.333333,5.0,5.555556,10.0,4.666667,7.0,2.500,4.0,0.0000,0.0,0.000,2.0,599b0563d9b4c2079007acc9,598e15296d8650eb27d52e3d,44.0,5.0,5.0,Prova 1 - Turma 01 - 2017.2,47.571429,7.0,7.0,Prova 2 - Turma 01 - 2017.2,11,25,12,6,4
4,20170032274,DAVI SIDRIM,2017.2,8.0,4.0,10.0,5.0,4.0,2.0,10.0,5.0,10.0,5.0,2.666667,4.0,3.888889,7.0,4.666667,7.0,4.375,7.0,5.4125,5.0,6.250,10.0,599b053410b11c07650122e9,598e15296d8650eb27d52e3d,32.0,4.0,5.0,Prova 1 - Turma 01 - 2017.2,80.857143,7.0,7.0,Prova 2 - Turma 01 - 2017.2,9,12,12,32,15


# Analise da tabela completa

*   Dado que voce fez x questoes na lista voce vai fazer quantas na prova
*   Dado que voce tirou x nos exercicios quanto voce vai tirar na prova
*   Dado que voce tirou y nos laboratorios quanto voce vai tirar na prova




